# Task 1: Data Collection and Preprocessing

## Overview
This notebook scrapes Google Play Store reviews for three Ethiopian banking apps and creates a clean, structured dataset.

## Target Apps
1. **Commercial Bank of Ethiopia (CBE)** - `com.combanketh.mobilebanking`
2. **Bank of Abyssinia (BOA)** - `com.boa.boaMobileBanking`
3. **Dashen Bank** - `com.cr2.amolelight`

## Objectives
- Scrape a minimum of 400 reviews per bank (1,200+ total)
- Clean and preprocess the collected reviews
- Save as `clean_reviews.csv`


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from google_play_scraper import app, reviews, Sort
from datetime import datetime
from tqdm import tqdm
import os
import sys

# Add scripts directory to path
sys.path.append(os.path.join('..', 'scripts'))

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## Configuration

Define the banking apps to scrape and set parameters.


In [2]:
# Configuration
BANK_APPS = {
    'CBE': {
        'name': 'Commercial Bank of Ethiopia',
        'app_id': 'com.combanketh.mobilebanking'
    },
    'BOA': {
        'name': 'Bank of Abyssinia',
        'app_id': 'com.boa.boaMobileBanking'
    },
    'Dashen': {
        'name': 'Dashen Bank',
        'app_id': 'com.cr2.amolelight'
    }
}

REVIEWS_PER_BANK = 400
LANG = 'en'
SORT = Sort.NEWEST

print(f"Target: {REVIEWS_PER_BANK} reviews per bank, Total: {REVIEWS_PER_BANK * len(BANK_APPS)} reviews")
print(f"Language: {LANG}, Sort: Newest first")


Target: 400 reviews per bank, Total: 1200 reviews
Language: en, Sort: Newest first


## Step 1: Scrape Reviews

Scrape reviews from Google Play Store for each banking app.


In [3]:
def scrape_reviews_for_app(app_id, app_name, count=400, lang='en', sort=Sort.NEWEST):
    """Scrape reviews for a specific app from Google Play Store."""
    print(f"\n{'='*60}")
    print(f"Scraping reviews for: {app_name}")
    print(f"App ID: {app_id}")
    print(f"{'='*60}")
    
    reviews_data = []
    continuation_token = None
    target_count = count
    batch_size = 200
    
    try:
        with tqdm(total=target_count, desc=f"Scraping {app_name}") as pbar:
            while len(reviews_data) < target_count:
                remaining = target_count - len(reviews_data)
                batch_count = min(batch_size, remaining)
                
                try:
                    result, continuation_token = reviews(
                        app_id,
                        lang=lang,
                        country='us',
                        sort=sort,
                        count=batch_count,
                        continuation_token=continuation_token
                    )
                    
                    reviews_data.extend(result)
                    pbar.update(len(result))
                    
                    if continuation_token is None or len(result) == 0:
                        print(f"\n⚠️  Only {len(reviews_data)} reviews available for {app_name}")
                        break
                        
                    import time
                    time.sleep(0.5)
                    
                except Exception as e:
                    print(f"\n❌ Error: {str(e)}")
                    break
        
        print(f"✅ Successfully scraped {len(reviews_data)} reviews for {app_name}")
        return reviews_data
        
    except Exception as e:
        print(f"\n❌ Failed to scrape reviews for {app_name}: {str(e)}")
        return []

# Scrape reviews for all banks
all_raw_reviews = {}

for bank_code, bank_info in BANK_APPS.items():
    app_id = bank_info['app_id']
    app_name = bank_info['name']
    
    raw_reviews = scrape_reviews_for_app(app_id, app_name, REVIEWS_PER_BANK)
    all_raw_reviews[app_name] = raw_reviews



Scraping reviews for: Commercial Bank of Ethiopia
App ID: com.combanketh.mobilebanking


Scraping Commercial Bank of Ethiopia: 100%|██████████| 400/400 [00:15<00:00, 25.13it/s]


✅ Successfully scraped 400 reviews for Commercial Bank of Ethiopia

Scraping reviews for: Bank of Abyssinia
App ID: com.boa.boaMobileBanking


Scraping Bank of Abyssinia: 100%|██████████| 400/400 [00:04<00:00, 80.53it/s] 


✅ Successfully scraped 400 reviews for Bank of Abyssinia

Scraping reviews for: Dashen Bank
App ID: com.cr2.amolelight


Scraping Dashen Bank: 100%|██████████| 400/400 [00:06<00:00, 62.01it/s]

✅ Successfully scraped 400 reviews for Dashen Bank


In [4]:
def preprocess_reviews(reviews_list, bank_name):
    """Clean and preprocess scraped reviews."""
    if not reviews_list:
        return pd.DataFrame()
    
    # Convert to DataFrame
    df = pd.DataFrame(reviews_list)
    
    # Select and rename relevant columns
    processed_df = pd.DataFrame({
        'review_text': df['content'].astype(str),
        'rating': df['score'].astype(int),
        'date': pd.to_datetime(df['at']).dt.date,
        'bank': bank_name,
        'source': 'Google Play'
    })
    
    # Remove duplicates
    initial_count = len(processed_df)
    processed_df = processed_df.drop_duplicates(subset=['review_text'], keep='first')
    duplicates_removed = initial_count - len(processed_df)
    
    if duplicates_removed > 0:
        print(f"  Removed {duplicates_removed} duplicate review(s)")
    
    # Drop missing values
    before_clean = len(processed_df)
    processed_df = processed_df.dropna(subset=['review_text', 'rating'])
    processed_df = processed_df[processed_df['review_text'].str.strip() != '']
    processed_df = processed_df[processed_df['review_text'] != 'nan']
    
    after_clean = len(processed_df)
    removed_missing = before_clean - after_clean
    
    if removed_missing > 0:
        print(f"  Removed {removed_missing} review(s) with missing/invalid data")
    
    # Standardize dates to YYYY-MM-DD
    processed_df['date'] = pd.to_datetime(processed_df['date']).dt.strftime('%Y-%m-%d')
    
    # Reset index
    processed_df = processed_df.reset_index(drop=True)
    
    return processed_df

# Preprocess all reviews
all_cleaned_dfs = []

for bank_name, raw_reviews in all_raw_reviews.items():
    print(f"\n📝 Preprocessing reviews for {bank_name}...")
    cleaned_df = preprocess_reviews(raw_reviews, bank_name)
    
    if not cleaned_df.empty:
        all_cleaned_dfs.append(cleaned_df)
        print(f"✅ Processed {len(cleaned_df)} clean reviews")



📝 Preprocessing reviews for Commercial Bank of Ethiopia...
  Removed 75 duplicate review(s)
✅ Processed 325 clean reviews

📝 Preprocessing reviews for Bank of Abyssinia...
  Removed 49 duplicate review(s)
✅ Processed 351 clean reviews

📝 Preprocessing reviews for Dashen Bank...
  Removed 76 duplicate review(s)
✅ Processed 324 clean reviews


In [5]:
# Combine all reviews
if all_cleaned_dfs:
    final_df = pd.concat(all_cleaned_dfs, ignore_index=True)
    
    # Remove any remaining duplicates across banks
    initial_total = len(final_df)
    final_df = final_df.drop_duplicates(subset=['review_text'], keep='first')
    final_duplicates = initial_total - len(final_df)
    
    if final_duplicates > 0:
        print(f"\n📊 Removed {final_duplicates} duplicate review(s) across all banks")
    
    # Ensure date format consistency
    final_df['date'] = pd.to_datetime(final_df['date']).dt.strftime('%Y-%m-%d')
    
    print(f"\n✅ Final dataset contains {len(final_df)} reviews")
    
    # Display preview
    print("\n📋 Dataset Preview:")
    display(final_df.head(10))
    
    print("\n📊 Dataset Info:")
    print(final_df.info())



📊 Removed 43 duplicate review(s) across all banks

✅ Final dataset contains 957 reviews

📋 Dataset Preview:


,review_text,rating,date,bank,source
0,It is nice app,5,2025-11-26,Commercial Bank of Ethiopia,Google Play
1,best,5,2025-11-25,Commercial Bank of Ethiopia,Google Play
2,good app,5,2025-11-25,Commercial Bank of Ethiopia,Google Play
3,it suddenly asked me to enter the verification...,1,2025-11-25,Commercial Bank of Ethiopia,Google Play
4,nice app...,5,2025-11-25,Commercial Bank of Ethiopia,Google Play
5,very good banking service and fast service,5,2025-11-24,Commercial Bank of Ethiopia,Google Play
6,thanks blc this app help me to use all time,5,2025-11-24,Commercial Bank of Ethiopia,Google Play
7,good,5,2025-11-24,Commercial Bank of Ethiopia,Google Play
8,why we can't send money to other people telebi...,1,2025-11-24,Commercial Bank of Ethiopia,Google Play
9,አታፍሩም ግን ለ15 birr 11.50 ብር ስትቆርጡ ይስተካከልልን,1,2025-11-23,Commercial Bank of Ethiopia,Google Play



📊 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 957 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_text  957 non-null    object
 1   rating       957 non-null    int64 
 2   date         957 non-null    object
 3   bank         957 non-null    object
 4   source       957 non-null    object
dtypes: int64(1), object(4)
memory usage: 44.9+ KB
None


## Step 4: Summary Statistics


In [6]:
# Summary statistics
print("="*60)
print("📊 DATASET SUMMARY")
print("="*60)
print(f"\nTotal reviews: {len(final_df)}")
print(f"\nReviews by bank:")
print(final_df['bank'].value_counts().to_string())
print(f"\nReviews by rating:")
print(final_df['rating'].value_counts().sort_index().to_string())
print(f"\nDate range: {final_df['date'].min()} to {final_df['date'].max()}")
print(f"\nMissing data: {final_df.isnull().sum().sum()} cells ({final_df.isnull().sum().sum() / (len(final_df) * len(final_df.columns)) * 100:.2f}%)")


📊 DATASET SUMMARY

Total reviews: 957

Reviews by bank:
bank
Bank of Abyssinia              333
Commercial Bank of Ethiopia    325
Dashen Bank                    299

Reviews by rating:
rating
1    243
2     41
3     69
4     71
5    533

Date range: 2023-04-08 to 2025-11-26

Missing data: 0 cells (0.00%)


## Step 5: Save to CSV

Save the cleaned dataset to `data/cleaned/clean_reviews.csv`.


In [7]:
# Save to CSV
output_path = os.path.join('..', 'data', 'cleaned', 'clean_reviews.csv')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

final_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Dataset saved to: {output_path}")
print(f"📁 File size: {os.path.getsize(output_path) / 1024:.2f} KB")

# Verify KPI targets
print("\n📈 KPI CHECK:")
total_reviews = len(final_df)
reviews_per_bank = final_df['bank'].value_counts()

print(f"✓ Total reviews: {total_reviews} (Target: 1,200+) {'✅' if total_reviews >= 1200 else '❌'}")
print(f"✓ Reviews per bank:")
for bank in BANK_APPS.values():
    count = reviews_per_bank.get(bank['name'], 0)
    status = "✅" if count >= 400 else "❌"
    print(f"  {status} {bank['name']}: {count} (Target: 400+)")

missing_data_pct = (final_df.isnull().sum().sum() / (len(final_df) * len(final_df.columns))) * 100
status = "✅" if missing_data_pct < 5 else "❌"
print(f"✓ Missing data: {missing_data_pct:.2f}% (Target: <5%) {status}")


✅ Dataset saved to: ..\data\cleaned\clean_reviews.csv
📁 File size: 103.33 KB

📈 KPI CHECK:
✓ Total reviews: 957 (Target: 1,200+) ❌
✓ Reviews per bank:
  ❌ Commercial Bank of Ethiopia: 325 (Target: 400+)
  ❌ Bank of Abyssinia: 333 (Target: 400+)
  ❌ Dashen Bank: 299 (Target: 400+)
✓ Missing data: 0.00% (Target: <5%) ✅
